![pipeline](pictures/pictures.002.png)

# Text-Fabric from ETCBC

This notebook assembles the data from the ETCBC that is needed
to compile its datasets in text-fabric-format on Github.
Ulltimately the data for the website [SHEBANQ](https://shebanq.ancient-data.org) will be
derived from these TF-sources.

## Pipeline
This is **pipe 1** of the pipeline from ETCBC data to the website SHEBANQ.

A run of this pipe produces a data *version*.
It should be run whenever there are new or updated data sources present that affect the output data.
Since all input data is delivered in a Github repo, we have excellent machinery to 
work with versioning.

The pipe works by executing a series of programs, contained in Github repositories.
For each repository in the pipe, a series of notebooks will be executed.
See [script mode](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) for 
details on how we call notebooks.

All this is specified in the configuration below.

### Core data

The core data is delivered by the ETCBC as `bhsa.mql.bz2` in 
the Github repo [bhsa](https://github.com/ETCBC/bhsa) in directory `source`.

This data will be converted by `tfFromMQL` in the `programs` directory.

The result of this action will be an updated TF resource in its 
`tf/core` directory.

### Additional data

Researchers have contributed to the dataset, 
but not all that data is in the core.
They are typically in the repository where the research has been 
executed, and where the data is documented.

Before the pipe starts, these repos must be pulled.

## Continuous version
Version `c` acts as a *continuous* version. It will be overwritten
by new snapshots of the data on a regular basis.

We support the following workflow to carry out these updates:

1. make a new version called `_temp`. Note:
   * this choice of name prevents it to reach github, because `_temp` directories are in `.gitignore`;
   * after running this workflow, the version `_temp` already exists, this is not a problem;
2. put a new data snapshot in the `source/_temp` directory of the `bhsa` repo, add also data to the
   `source/_temp` directories of the other repos in the pipeline, as far as relevant;
3. run `good = runPipeline(pipeline, versions=['_temp'], force=True)`. Note:
   * we use `force=True` here, because then the old data in version `_temp` will be thoroughly overwritten;
4. if all went well run `copyVersion(pipeline, '_temp', 'c')`. This will overwrite all data directories
   in version `c` by the just created data directories in `temp`.

In [1]:
import os,sys,collections
from pipeline import runPipeline, copyVersion
from tf.fabric import Fabric

# Config

In [2]:
CORE_NAME = 'bhsa'

if 'SCRIPT' not in locals(): 
    SCRIPT = False
    DEFAULT_CORE_NAME = CORE_NAME
    DEFAULT_VERSION = 'c'

# Pipeline settings

Here all the nitty gritty differences between versions are stated.

In [3]:
pipeline = dict(
    defaults = dict(
        CORE_NAME=CORE_NAME,
        VERSION=DEFAULT_VERSION,
        LANG_FEATURE='language',
        OCC_FEATURE='g_cons',
        LEX_FEATURE='lex',
        TEXT_FEATURE='g_word_utf8',
        TRAILER_FEATURE='trailer_utf8',
        DO_VOCALIZED_LEXEME=True,
        EXTRA_OVERLAP='',
        LEX_FORMATS='@fmt:lex-trans-plain={lex0} ',
        RENAME=(
            ('g_suffix', 'trailer'),
            ('g_suffix_utf8', 'trailer_utf8'),
        ),
    ),
    versions={
        '_temp': dict(),
        'c': dict(),
        '2017': dict(),
        '2016': dict(),
        '4b': dict(
            DO_VOCALIZED_LEXEME=False,
            EXTRA_OVERLAP='gloss nametype',
            LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
            ),
        '4': dict(
            DO_VOCALIZED_LEXEME=False,
            EXTRA_OVERLAP='gloss nametype',
            LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
        ),
        '3': dict(
                LANG_FEATURE='language',
                OCC_FEATURE='surface_consonants',
                LEX_FEATURE='lexeme',
                TEXT_FEATURE='text',
                TRAILER_FEATURE='suffix',
                LEX_FORMAT='@fmt:lex-trans-plain={lexeme} ',
            ),
    },
    repoOrder = '''
        bhsa
        phono
        valence
        parallels
    ''',
    repoConfig = dict(
        bhsa=(
            dict(
                task='coreData',
            ),
            dict(
                task='bookNames',
                omit={},
            ),
            dict(
                task='lexicon',
                omit={'3'},
            ),
            dict(
                task='paragraphs',
                 omit={'3', '4', '4b'},
            ),
            dict(
                task='ketivQere',
                omit={'3', '4', '4b'},
            ),
            dict(
                task='stats',
                omit={'4', '4b'},
            ),
        ),
        phono=(
            dict(
                task='phono',
                omit={'3', '4', '4b'},
            ),
        ),
        valence=(
            dict(
                task='enrich',
                omit={'3'},
            ),
            dict(
                task='flowchart',
                omit={'3'},
            ),
        ),
        parallels=(
            dict(
                task='parallels',
                omit={},
                params=dict(
                    FORCE_MATRIX=False,
                ),
            ),
        ),
    ),
    repoDataDirs = dict(
        bhsa      = 'source _temp tf shebanq',
        phono     = '_temp tf',
        valence   = 'source _temp tf shebanq',
        parallels = 'source _temp tf',
    ),
)

# Run the pipeline

In [4]:
good = runPipeline(pipeline, versions=['_temp'], force=True)


##############################################################################################
#                                                                                            #
#       0.00s Make version [_temp]                                                           #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*       0.00s Make repo [bhsa]                                                               *
*                                                                                            *
**********************************************************************************************


---------------------------------------------

  0.25s 			feature kind (str) =def= unknown : node
  0.25s 			feature rela (str) =def= NA : node
  0.25s 			feature mother_object_type (str) =def= clause : node
  0.26s 			feature dist_unit (str) =def= clause_atoms : node
  0.26s 		otype half_verse
  0.26s 			feature label (str) =def=  : node
  0.26s 		otype verse
  0.27s 			feature verse (int) =def= 0 : node
  0.27s 			feature chapter (int) =def= 0 : node
  0.27s 			feature label (str) =def=  : node
  0.27s 			feature book (str) =def= Genesis : node
  0.27s 		otype phrase_atom
  0.27s 			feature number (int) =def= 0 : node
  0.27s 			feature dist (int) =def= 0 : node
  0.28s 			feature distributional_parent (str) =def= 0 : edge
  0.28s 			feature mother (str) =def= 0 : edge
  0.28s 			feature functional_parent (str) =def= 0 : edge
  0.28s 			feature det (str) =def= NA : node
  0.28s 			feature typ (str) =def= VP : node
  0.28s 			feature rela (str) =def= NA : node
  0.29s 			feature dist_unit (str) =def= clause_atoms : node
  0.29s 		

   |     0.73s T nme                  to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.75s T nu                   to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     2.15s T number               to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.61s T otype                to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.74s T pdp                  to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.75s T pfm                  to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.77s T prs                  to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.79s T prs_gn               to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.80s T prs_nu               to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.79s T prs_ps               to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.75s T ps                   to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.68s T qere                 to /Users/dirk/g

 1m 29s All features loaded/computed - for details use loadLog()
..............................................................................................
.      6m 07s Load and compile all other TF features                                         .
..............................................................................................
   |     0.00s Feature overview: 70 for nodes; 4 for edges; 1 configs; 7 computed
  0.00s loading features ...
   |     0.18s T code                 from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     1.76s T det                  from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     1.49s T dist                 from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     2.00s T dist_unit            from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     3.83s T distributional_parent from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.27s T domain               from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.83s T function             from /Us

75 features found and 0 ignored
  0.00s loading features ...
   |     0.01s B book                 from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.00s Feature overview: 70 for nodes; 4 for edges; 1 configs; 7 computed
  4.38s All features loaded/computed - for details use loadLog()
|      7m 29s 26 book name features created
..............................................................................................
.      7m 29s Write book name features as TF                                                 .
..............................................................................................
   |     0.00s T book@am              to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.00s T book@ar              to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.00s T book@bn              to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.00s T book@da              to /Users/dirk/github/etcbc/bhsa/_temp/_temp/tf
   |     0.00s T book@de             

...

|     11m 43s 	Destination /Users/dirk/github/etcbc/valence/tf/_temp/.tf/valence.tfx exists
|     11m 43s NOTE: repo seems up to date. Will be run because of "force=True"
True True
..............................................................................................
.     11m 43s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.0.6
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

113 features found and 0 ignored
  0.00s loading features ...
   |     0.22s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.17s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.01s B gloss                from /Users/dirk/github/et

|     12m 30s 	Done
|     12m 30s 	Phrases of kind C :  19302
|     12m 30s 	Phrases of kind L :  11680
|     12m 30s 	Phrases of kind I :   6018
|     12m 30s 	Total complements :  37000
|     12m 30s 	Total phrases     : 214541
..............................................................................................
.     12m 30s Checking enrichment logic                                                      .
..............................................................................................
|     12m 30s 	All 6 rules OK
..............................................................................................
.     12m 30s Generating enrichments                                                         .
..............................................................................................
|     12m 37s 	Generated enrichment values for 1380 verbs:
|     12m 37s 	Enriched values for 221366 nodes
|     12m 37s 	Overview of rule applications:
|     12m 37s gen

|     12m 47s valence                   ... differences after the metadata
|     12m 48s 	line      2 OLD -->427558	complement<--
|     12m 48s 	line      2 NEW -->427561	complement<--
|     12m 48s 	line      3 OLD -->427584	complement<--
|     12m 48s 	line      3 NEW -->427587	complement<--
|     12m 48s 	line      4 OLD -->427596	complement<--
|     12m 48s 	line      4 NEW -->427599	complement<--
|     12m 48s 	line      5 OLD -->427602	NA<--
|     12m 48s 	line      5 NEW -->427605	NA<--

|     12m 48s Done
..............................................................................................
.     12m 48s Deliver features to /Users/dirk/github/etcbc/valence/tf/_temp                  .
..............................................................................................
|     12m 48s 	valence
|     12m 48s 	predication
|     12m 48s 	grammatical
|     12m 48s 	original
|     12m 48s 	lexical
|     12m 48s 	semantic
|     12m 48s 	f_correction
|     12m 48s 	s_man

|     13m 16s 	Done, 47575 clauses with relevant constituents
..............................................................................................
.     13m 16s Counting constituents                                                          .
..............................................................................................
|     13m 17s 	22384 clauses with  1 dos        constituents
|     13m 17s 	25191 clauses with  0 dos        constituents
|     13m 17s 	22384 clauses with  a dos        constituent
|     13m 17s 	 3554 clauses with  1 pdos       constituents
|     13m 17s 	44021 clauses with  0 pdos       constituents
|     13m 17s 	 3554 clauses with  a pdos       constituent
|     13m 17s 	  989 clauses with  1 ndos       constituents
|     13m 17s 	46586 clauses with  0 ndos       constituents
|     13m 17s 	  989 clauses with  a ndos       constituent
|     13m 17s 	  111 clauses with  1 kdos       constituents
|     13m 17s 	47464 clauses with  0 kdos     

|     13m 48s 	Destination /Users/dirk/github/etcbc/parallels/tf/_temp/.tf/crossref.tfx exists
|     13m 48s NOTE: repo seems up to date. Will be run because of "force=True"
..............................................................................................
.     13m 48s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.0.6
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

113 features found and 0 ignored
  0.00s loading features ...
   |     0.03s B otype                from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.01s B book                 from /Users/dirk/github/etcbc/bhsa/tf/_temp
   |     0.01s B chapter              from /Users/dirk/github/etcbc/bhs

# Consolidate the continuous version
If you have run an update version called `_temp`, and all has went well
you can copy over the entire version (including its source and temp directories to `c`).
This will happen for all repos in the pipeline.

In [5]:
#good = True
#good = False

if good:
    copyVersion(pipeline, '_temp', 'c')


##############################################################################################
#                                                                                            #
#     14m 11s Copy version _temp ==> c                                                       #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*     14m 11s Repo bhsa                                                                      *
*                                                                                            *
**********************************************************************************************

|     14m 11s 	Copy source/_temp ==> source/c
